In [ ]:
from  psaw import PushshiftAPI
import datetime
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import pandas_datareader as web
from dateutil import relativedelta
from tabulate import tabulate

api = PushshiftAPI() #will be used to access reddit

NASDAQ_df = pd.read_csv('NASDAQ.csv') #import ticker data
AMEX_df = pd.read_csv('AMEX.csv')
NYSE_df = pd.read_csv('NYSE.csv')

frames = [NASDAQ_df, AMEX_df, NYSE_df] #combine the ticker data into a dataframe
temp = pd.concat(frames)
cols = list(temp.columns)
temp.drop(cols[2:], axis = 1, inplace = True)
tickers = temp['Symbol'].tolist()


def stonk_mentions(start_time, end_time, tickers, m = 1): #function that searches the r/wallstreetbets 
                                                        #subreddit between specified time periods
    start_time_int = int(start_time.timestamp())        #returns the tickers mentioned and how many posts mentioned them
    end_time_int = int(end_time.timestamp())
    next_month_time = end_time + relativedelta.relativedelta(months=m)
    
    submissions = api.search_submissions(after = start_time_int, before = end_time_int,
                                subreddit = 'wallstreetbets',
                                filter = ['url', 'author', 'title', 'subreddit'])
    
    df = pd.DataFrame({"Ticker" : tickers}) #pulls in list of tickers to cross reference
    
    mentions = [] #stores each individual mention
    for submission in submissions:
        words = submission.title.split()
        cashtags = list(set(filter(lambda word: word.upper().startswith('$'), words))) #function identifies cashtags
        
        if len(cashtags) > 0: #if a cashtage is identified in each submission title it pulls out the ticker 
            for cashtag in cashtags:
                tick = cashtag.replace('$','')
                if tick in tickers:
                    mentions.append(tick)
    
    occurances = []
    for tick in tickers:
        try: 
            occurances.append(mentions.count(tick.upper())) #counts the number of times each ticker in our list 
        except:                                             #is mentioned
            pass 
        
    df['Mentions'] = occurances 
    df['Start_Time'] = start_time 
    df['End_Time'] = end_time
    df2 = df[df['Mentions'] > 20] #creates dataframe for tickers over 20 mentions
    df2 = df2.sort_values('Mentions', axis = 0, ascending = False)
    return df, df2


t = datetime.datetime(2020,11,30) #runs the function for weekly intervals starting at this date
dates = [t]
for i in range(1,53):
    dates.append(t+relativedelta.relativedelta(weeks=i)) #12 months of time history
    
frames_df = []
frames_df2 = []
for i in range(len(dates)):
    df, df2 = stonk_mentions(dates[i], dates[i+1], tickers, 1) 
    frames_df.append(df) #combines each outputted dataframe
    frames_df2.append(df2) 
    print(i)
    
frames_df2.reset_index(inplace = True) #reset index as we are working with this dataframe

df = frames_df2

returns_0w_list = []
returns_1w_list = []
vol_0w_list = []
vol_1w_list = []

for i in range(len(df)): #for the dataframe that we are working with this function gets the returns needed 
    start_time = datetime.datetime.strptime(df['Start_Time'][i], '%Y-%m-%d') #for the 0 week and 1 week strategies
    end_time = datetime.datetime.strptime(df['End_Time'][i], '%Y-%m-%d')
    next_time = end_time + relativedelta.relativedelta(weeks=1)
    try:
        price_0w = web.get_data_yahoo(df['Ticker'][i],
                        start = start_time,
                        end = end_time)['Adj Close']
        returns_0w_list.append( (price_0w[-1]/price_0w[0]) - 1 )
        vol_0w_list.append( np.std(price_0w.pct_change()) ) 

        price_1w = web.get_data_yahoo(df['Ticker'][i],
                        start = end_time,
                        end = next_time)['Adj Close']
        returns_1w_list.append( (price_1w[-1]/price_1w[0]) - 1 )
        vol_1w_list.append( np.std(price_1w.pct_change()) )
    except:
        returns_0w_list.append(0)
        returns_1w_list.append(0)
        vol_0w_list.append(0)
        vol_1w_list.append(0)
        pass 
    
    
df['Returns_0w'] = returns_0w_list
df['Returns_1w'] = returns_1w_list
df['Vol_0w'] = vol_0w_list
df['Vol_1w'] = vol_1w_list

df = df[['Ticker', 'Mentions', #final dataframe we are working with
       'Start_Time', 'End_Time', 'Returns_0w', 'Returns_1w', 'Vol_0w',
       'Vol_1w']]

df[['Mentions','Returns_0w','Returns_1w','Vol_0w','Vol_1w']].corr() 

df = df.sort_values(by=["Start_Time", 'Mentions'])

t = datetime.datetime(2020,11,30)
dates = [t]
for i in range(1,53):
    dates.append(t+relativedelta.relativedelta(weeks=i))
    
weekly_returns0w = [1, ]
weekly_returns1w = [1, 1]
SP500_returns = [1, ]

for date in dates[:-1]: #calculates portfolio strategy and the S&P 500 returns
    return0w = []
    return1w = []
#     start_time = datetime.datetime.strptime(df['Start_Time'][i], '%Y-%m-%d')

    end_time = date + relativedelta.relativedelta(weeks=1)
    price_0w = web.get_data_yahoo('SPY',
                        start = date,
                        end = end_time)['Adj Close']
    SP500_returns.append((price_0w[-1]/price_0w[0]))
    for i in range(len(df['Start_Time'])):
        if dt.strptime(df['Start_Time'][i], "%Y-%m-%d").date() == date.date():
            return0w.append(df['Returns_0w'][i] + 1)
            return1w.append(df['Returns_1w'][i]+ 1)
    
    if len(return0w) == 0:
        weekly_returns0w.append(1)
    else:
        weekly_returns0w.append(sum(return0w)/len(return0w))
    
    if len(return1w) == 0:
        weekly_returns1w.append(1)
    else:       
        weekly_returns1w.append(sum(return1w)/len(return1w))     
            
            
weekly_returns0w_cumulative_return = np.cumprod(weekly_returns0w) #calculates the cumulative return
weekly_returns1w_cumulative_return = np.cumprod(weekly_returns1w[:-1])
SP500_returns_cumulative_return = np.cumprod(SP500_returns)

#graphs of results
plt.plot(dates, weekly_returns0w_cumulative_return, label = '0 Week Strategy', color = 'blue')
plt.plot(dates, SP500_returns_cumulative_return, label = 'S&P 500', color = 'red')
plt.legend()
plt.show()

plt.plot(dates, weekly_returns1w_cumulative_return, label = '1 Week Strategy', color = 'green')
plt.plot(dates, SP500_returns_cumulative_return, label = 'S&P 500', color = 'red')
plt.legend()
plt.show()

df_cumreturns = pd.DataFrame({'Dates': dates,
                              'S&P500 Weekly Return': SP500_returns,
                              'S&P500 Cumulative Return' : SP500_returns_cumulative_return,
                              '+0 Weekly Return': weekly_returns0w,
                              '+0 Cumulative Return': weekly_returns0w_cumulative_return,
                              '+1 Weekly Return': weekly_returns1w[:-1],
                              '+1 Cumulative Return': weekly_returns1w_cumulative_return,
                             })

table = [['Portfolio', 'Total Return', 'Volatility'], #outputs the final table of results
    ['S&P 500', "{:.2%}".format(-1 + float(df_cumreturns['S&P500 Cumulative Return'].tail(1))), "{:.2%}".format(np.std(df_cumreturns['S&P500 Weekly Return']))],
    ['Week 0 Strategy', "{:.2%}".format(-1 + float(df_cumreturns['+0 Cumulative Return'].tail(1))), "{:.2%}".format(np.std(df_cumreturns['+0 Weekly Return']))],
    ['Week 1 Strategy', "{:.2%}".format(-1 + float(df_cumreturns['+1 Cumulative Return'].tail(1))), "{:.2%}".format(np.std(df_cumreturns['+1 Weekly Return']))],
        ]
print(tabulate(table, headers='firstrow'))